In [81]:
## import pandas 
import pandas as pd 

In [83]:
## let's read the file 
df = pd.read_excel("raw-data/Demographic_Snapshot_ENL.xlsx", sheet_name = "School")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9363 entries, 0 to 9362
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   DBN                             9363 non-null   object 
 1   School Name                     9363 non-null   object 
 2   Year                            9363 non-null   object 
 3   Total Enrollment                9363 non-null   int64  
 4   Grade 3K                        9363 non-null   int64  
 5   Grade PK (Half Day & Full Day)  9363 non-null   int64  
 6   Grade K                         9363 non-null   int64  
 7   Grade 1                         9363 non-null   int64  
 8   Grade 2                         9363 non-null   int64  
 9   Grade 3                         9363 non-null   int64  
 10  Grade 4                         9363 non-null   int64  
 11  Grade 5                         9363 non-null   int64  
 12  Grade 6                         93

In [91]:
## Let's now test the data for any relevant duplicates 
# Clean School Name for consistency
df["School Name"] = (
    df["School Name"]
    .str.lower()
    .str.strip()
    .str.replace(r'[^\w\s]', '', regex=True)
)

# Clean Year column by stripping whitespace
df["Year"] = df["Year"].str.strip()

# Count duplicates for each School Name + Year pair (for overview)
duplicate_counts = df.groupby(["School Name", "Year"]).size().reset_index(name="counts")
duplicates_in_school_year = duplicate_counts[duplicate_counts["counts"] > 1]

print(f"Number of duplicated school-year entries (by School Name): {duplicates_in_school_year.shape[0]}")
print(duplicates_in_school_year)

# Find true duplicate records by DBN + Year (exact duplicates)
duplicates = df[df.duplicated(subset=["DBN", "Year"], keep=False)]

print(f"Number of true duplicate DBN-year records: {duplicates.shape[0]}")
print(duplicates)

Number of duplicated school-year entries (by School Name): 20
                                            School Name     Year  counts
3852  new visions charter high school for advanced math  2019-20       4
3853  new visions charter high school for advanced math  2020-21       4
3854  new visions charter high school for advanced math  2021-22       4
3855  new visions charter high school for advanced math  2022-23       4
3856  new visions charter high school for advanced math  2023-24       4
3857  new visions charter high school for the humani...  2019-20       3
3858  new visions charter high school for the humani...  2020-21       3
3859  new visions charter high school for the humani...  2021-22       3
3860  new visions charter high school for the humani...  2022-23       3
3861  new visions charter high school for the humani...  2023-24       3
6974                                             ps 360  2019-20       2
6975                                             ps 360  2020-

In our analysis, we found that while some school names appear multiple times with different DBN codes (unique identifiers), there are no true duplicates when considering the combination of DBN and Year. Since no DBN-Year pairs are repeated, our dataset is confirmed to be accurate and reliable for further use.

In [93]:
## let's eliminate unwanted columns 
df_clean = df[["School Name", "Year", "Total Enrollment", "# English Language Learners", "% English Language Learners"]]
df_clean

,School Name,Year,Total Enrollment,# English Language Learners,% English Language Learners
0,ps 015 roberto clemente,2019-20,190,17,0.089474
1,ps 015 roberto clemente,2020-21,193,21,0.108808
2,ps 015 roberto clemente,2021-22,179,11,0.061453
3,ps 015 roberto clemente,2022-23,180,12,0.066667
4,ps 015 roberto clemente,2023-24,189,24,0.126984
...,...,...,...,...,...
9358,bronx charter school for the arts,2019-20,523,69,0.131931
9359,bronx charter school for the arts,2020-21,626,78,0.124601
9360,bronx charter school for the arts,2021-22,598,79,0.132107
9361,bronx charter school for the arts,2022-23,591,73,0.123519


In [97]:
## to improve readability, let's define each year range using the column name 'Start Year' 
df_clean['Start Year'] = df_clean['Year'].str.extract(r'(\d{4})').astype(int)
df_clean

/var/folders/6p/9zpd5gjx1dj6zpc834rhv6d40000gn/T/ipykernel_98100/612969100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Start Year'] = df_clean['Year'].str.extract(r'(\d{4})').astype(int)


,School Name,Year,Total Enrollment,# English Language Learners,% English Language Learners,Start Year
0,ps 015 roberto clemente,2019-20,190,17,0.089474,2019
1,ps 015 roberto clemente,2020-21,193,21,0.108808,2020
2,ps 015 roberto clemente,2021-22,179,11,0.061453,2021
3,ps 015 roberto clemente,2022-23,180,12,0.066667,2022
4,ps 015 roberto clemente,2023-24,189,24,0.126984,2023
...,...,...,...,...,...,...
9358,bronx charter school for the arts,2019-20,523,69,0.131931,2019
9359,bronx charter school for the arts,2020-21,626,78,0.124601,2020
9360,bronx charter school for the arts,2021-22,598,79,0.132107,2021
9361,bronx charter school for the arts,2022-23,591,73,0.123519,2022


In [99]:
## Now, we'll generate a pivot table with school years as column values

school_ell_trends = df_clean.pivot_table(
    index='School Name',
    columns='Start Year',
    values='# English Language Learners'
)
school_ell_trends

Start Year,2019,2020,2021,2022,2023
School Name,,,,,
47 the american sign language and english secondar,11.0,15.0,9.0,8.0,12.0
a philip randolph campus high school,185.0,223.0,216.0,176.0,195.0
a school without walls,NaN,NaN,NaN,NaN,3.0
abraham lincoln high school,251.0,272.0,278.0,309.0,321.0
academic leadership charter school,75.0,83.0,73.0,52.0,46.0
...,...,...,...,...,...
young womens leadership school queens,10.0,18.0,15.0,26.0,42.0
zeta charter school mount eden,NaN,4.0,44.0,66.0,93.0
zeta charter school tremont park,NaN,3.0,24.0,37.0,37.0


In [57]:
school_ell_trends.head()

Start Year,2019,2020,2021,2022,2023
School Name,,,,,
47 the american sign language and english secondar,11.0,NaN,NaN,NaN,NaN
a philip randolph campus high school,185.0,NaN,NaN,NaN,NaN
a school without walls,NaN,NaN,NaN,NaN,3.0
abraham lincoln high school,251.0,NaN,NaN,NaN,NaN
academic leadership charter school,75.0,NaN,NaN,NaN,NaN


In [101]:
# How many unique schools are in the filtered DataFrame?
# let's check to see the data is accurate 
original_school_count = df_clean['School Name'].nunique()
original_school_count

1886

In [103]:
# How many schools made it into the pivot table?
# let's check to see the data is accurate 
pivot_school_count = school_ell_trends.shape[0]
pivot_school_count

1886

In [105]:
## Let's replace the NaN values with 0 
school_ell_trends = school_ell_trends.fillna(0)
school_ell_trends

Start Year,2019,2020,2021,2022,2023
School Name,,,,,
47 the american sign language and english secondar,11.0,15.0,9.0,8.0,12.0
a philip randolph campus high school,185.0,223.0,216.0,176.0,195.0
a school without walls,0.0,0.0,0.0,0.0,3.0
abraham lincoln high school,251.0,272.0,278.0,309.0,321.0
academic leadership charter school,75.0,83.0,73.0,52.0,46.0
...,...,...,...,...,...
young womens leadership school queens,10.0,18.0,15.0,26.0,42.0
zeta charter school mount eden,0.0,4.0,44.0,66.0,93.0
zeta charter school tremont park,0.0,3.0,24.0,37.0,37.0


In [107]:
## Percentage change between (2021 - 2022) and (2022 - 2023)
school_ell_trends['Change_2021_2022'] = ((school_ell_trends[2022] - school_ell_trends[2021]) / school_ell_trends[2021]) * 100

## Percentage change between (2022 - 2023) and (2023 - 2024)
school_ell_trends['Change_2022_2023'] = ((school_ell_trends[2023] - school_ell_trends[2022]) / school_ell_trends[2022]) * 100

## Display updated DataFrame
school_ell_trends

Start Year,2019,2020,2021,2022,2023,Change_2021_2022,Change_2022_2023
School Name,,,,,,,
47 the american sign language and english secondar,11.0,15.0,9.0,8.0,12.0,-11.111111,50.000000
a philip randolph campus high school,185.0,223.0,216.0,176.0,195.0,-18.518519,10.795455
a school without walls,0.0,0.0,0.0,0.0,3.0,NaN,inf
abraham lincoln high school,251.0,272.0,278.0,309.0,321.0,11.151079,3.883495
academic leadership charter school,75.0,83.0,73.0,52.0,46.0,-28.767123,-11.538462
...,...,...,...,...,...,...,...
young womens leadership school queens,10.0,18.0,15.0,26.0,42.0,73.333333,61.538462
zeta charter school mount eden,0.0,4.0,44.0,66.0,93.0,50.000000,40.909091
zeta charter school tremont park,0.0,3.0,24.0,37.0,37.0,54.166667,0.000000


In [109]:
## Filter out rows where the base year is zero or where the change is infinite
valid_2021_2022 = school_ell_trends['Change_2021_2022'].replace([float('inf'), -float('inf')], pd.NA).dropna()
valid_2022_2023 = school_ell_trends['Change_2022_2023'].replace([float('inf'), -float('inf')], pd.NA).dropna()

## Calculate mean of the cleaned columns
avg_change_2021_2022 = valid_2021_2022.mean()
avg_change_2022_2023 = valid_2022_2023.mean() 

In [111]:
#average rate of change between 2021-2022 and 2022-2023 where you saw the beginning of the immigration influx
avg_change_2021_2022

8.634362922908755

In [113]:
#average rate of change between 2022-2023 and 2023-2024 where you saw an ongoing increase of immigrant children 
avg_change_2022_2023

25.659581642469096

In [115]:
##Highest to lowest based on the percent change in ENL enrollment between 2022 and 2023
import numpy as np

## Replace 0 with np.nan in 2022 before calculating percent change
school_ell_trends['Change_2022_2023'] = (
    (school_ell_trends[2023] - school_ell_trends[2022].replace(0, np.nan)) /
    school_ell_trends[2022].replace(0, np.nan)
) * 100

## Drop rows with NaN or inf in the change column
school_ell_trends_cleaned = school_ell_trends.replace([np.inf, -np.inf], np.nan).dropna(subset=['Change_2022_2023'])

## Sort from highest to lowest
school_ell_trends_sorted = school_ell_trends_cleaned.sort_values(by='Change_2022_2023', ascending=False)

## View top 10 results
school_ell_trends_sorted.head(11)

Start Year,2019,2020,2021,2022,2023,Change_2021_2022,Change_2022_2023
School Name,,,,,,,
ps 064 robert simon,4.0,4.0,3.0,2.0,25.0,-33.333333,1150.000000
ps 306 ethan allen,4.0,2.0,3.0,2.0,22.0,-33.333333,1000.000000
ps 333 manhattan school for children,16.0,12.0,11.0,7.0,67.0,-36.363636,857.142857
olympus academy,4.0,3.0,3.0,1.0,8.0,-66.666667,700.000000
the riverside school for makers and artists,23.0,29.0,21.0,24.0,182.0,14.285714,658.333333
lower manhattan community middle school,17.0,20.0,14.0,6.0,42.0,-57.142857,600.000000
university neighborhood middle school,11.0,8.0,5.0,4.0,27.0,-20.000000,575.000000
the peck slip school,8.0,8.0,13.0,12.0,80.0,-7.692308,566.666667
mosaic preparatory academy,20.0,20.0,12.0,11.0,70.0,-8.333333,536.363636


In [117]:
# Now let's do total enrollment per year
total_enrollment = school_ell_trends[[2019, 2020, 2021, 2022, 2023]].sum()
total_enrollment

Start Year
2019    141529.583333
2020    145869.500000
2021    146285.583333
2022    148040.916667
2023    162951.666667
dtype: float64

In [119]:
school_ell_trends_sorted.head(26)

Start Year,2019,2020,2021,2022,2023,Change_2021_2022,Change_2022_2023
School Name,,,,,,,
ps 064 robert simon,4.0,4.0,3.0,2.0,25.0,-33.333333,1150.000000
ps 306 ethan allen,4.0,2.0,3.0,2.0,22.0,-33.333333,1000.000000
ps 333 manhattan school for children,16.0,12.0,11.0,7.0,67.0,-36.363636,857.142857
olympus academy,4.0,3.0,3.0,1.0,8.0,-66.666667,700.000000
the riverside school for makers and artists,23.0,29.0,21.0,24.0,182.0,14.285714,658.333333
lower manhattan community middle school,17.0,20.0,14.0,6.0,42.0,-57.142857,600.000000
university neighborhood middle school,11.0,8.0,5.0,4.0,27.0,-20.000000,575.000000
the peck slip school,8.0,8.0,13.0,12.0,80.0,-7.692308,566.666667
mosaic preparatory academy,20.0,20.0,12.0,11.0,70.0,-8.333333,536.363636
